# DLCD itteration comparison tool 

In [2]:
'''
landcover classification legend

number 	landcover type 
0	no data
1	Extraction Sites
3	Inland Waterbodies
4	Salt Lakes
5	Irrigated Cropping
6	Irrigated Pasture
7	Irrigated Sugar
8	Rainfed Cropping
9	Rainfed Pasutre
10	Rainfed Sugar
11	Wetlands
14	Tussock Grasses - Closed
15	Alpine Grasses - Open
16	Hummock Grasses - Open
18	Tussock Grasses - Open
19	Shrubs and Grasses - Sparse-Scattered
24	Shrubs - Closed
25	Shrubs - Open
31	Trees - Closed
32	Trees - Open
33	Trees - Scattered
34	Trees - Sparse
35	Built up area '''


'\nlandcover classification legend\n\nnumber \tlandcover type \n0\tno data\n1\tExtraction Sites\n3\tInland Waterbodies\n4\tSalt Lakes\n5\tIrrigated Cropping\n6\tIrrigated Pasture\n7\tIrrigated Sugar\n8\tRainfed Cropping\n9\tRainfed Pasutre\n10\tRainfed Sugar\n11\tWetlands\n14\tTussock Grasses - Closed\n15\tAlpine Grasses - Open\n16\tHummock Grasses - Open\n18\tTussock Grasses - Open\n19\tShrubs and Grasses - Sparse-Scattered\n24\tShrubs - Closed\n25\tShrubs - Open\n31\tTrees - Closed\n32\tTrees - Open\n33\tTrees - Scattered\n34\tTrees - Sparse\n35\tBuilt up area '

In [3]:
#load requiered modules 
import numpy as np
import rasterio
#from matplotlib import pyplot as plt #don't need unless want to plot
import xarray as xr
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import os.path
from scipy import ndimage
from scipy.ndimage import measurements
from scipy import stats

import dask.array as da

#set up filepath due to folder re-arrangement 
os.chdir('/g/data/r78/ck9738')


In [4]:
#define nessisary gotiff handeling fuctions

import xarray, rasterio, numpy as np
def numpy_to_xarray(array, geobox, name=None):
    """Utility to convert ndarray to DataArray, using a datacube.model.GeoBox"""
    coords=[xarray.Coordinate(x, geobox.coords[x].values, attrs=dict(units=geobox.coords[x].units)) 
            for x in geobox.dims]
    return xarray.DataArray(array, coords=coords, attrs=dict(crs=geobox.crs), name=name)

def geopandas_to_xarray(table, geobox, name=None):
    """Rasterise (with reprojection)"""
    array = rasterio.features.rasterize(shapes=table.to_crs(geobox.crs._crs.ExportToProj4()).geometry,
                                        out_shape=(geobox.height, geobox.width),
                                        transform=geobox.affine)
    return numpy_to_xarray(array, geobox, name)

def rasterfile_to_xarray(file, geobox, name=None, nodata=True, num_bands=(1), X_band= 1):
    """Blit like
    """
    with rasterio.open(file) as src:

        assert src.indexes == num_bands # assume single band
        band = rasterio.band(src, X_band) # do not attempt to read entire extent into memory
        array = np.empty((geobox.height, geobox.width), dtype=band.dtype)
        
        rasterio.warp.reproject(source=band,
                                destination=array,
                                dst_crs=geobox.crs.crs_str,
                                dst_transform=geobox.affine,
                                dst_nodata=nodata)
    return numpy_to_xarray(array, geobox, name)

def load_gfif_xr(file_name, x=1):


    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    data_xr = xr.DataArray(data_array) #turn into Xr array for fuctionality #Now don't need xr yet
    return(data_xr) #return np array into program 

def load_gfif_da(file_name, x=1):


    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    data_da = da.from_array(data_array, chunks=(20,20)) #turn into dask  array for fuctionality 
    return(data_da) #return da array into program 

In [5]:

old_version = load_gfif_xr('old_DLCD/DLCDv2_2010_20111.tif',1)#for old dlcd files all only have ONE band (are single year)

new_version = load_gfif_xr('new_DLCD/DLCDv2_2016_new.tif',10)#for new DLCD band number corrosponds to year in following table

In [6]:
# define function to return the number of pixles which are class A in old and class B in new:
#fuction returns an array, int: array with 1= occurance of class overlap and 0= everything else.

def count_overlap_array(old_np,A,new_np,B):
    raw_np = np.ones_like(old_np) #generate a np with the same size as the dlcd but full of 1 values
    raw_xr = xr.DataArray(raw_np) #turn to xarray for it's useful fuctionality
    count_old = raw_xr.where(old_np==A) # make an xr where 1= presance of class A, 0= absence 
    count_new = count_old.where(new_np==B) # make an xr where 1= overlap of classes A&B
    count_class = count_new.fillna(0) #replaces all the nan with 0
    howmuch_name = np.count_nonzero(count_class) #counts the number of non 0 in the xaray


    return(count_class) #returns an array

In [7]:
#find class change 

this_change = count_overlap_array(old_version ,19,new_version,3)

In [9]:
old_version

<xarray.DataArray (dim_0: 14902, dim_1: 19161)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)
Dimensions without coordinates: dim_0, dim_1

In [8]:
this_change

<xarray.DataArray (dim_0: 14902, dim_1: 19161)>
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])
Dimensions without coordinates: dim_0, dim_1

In [9]:
#Create output list of points where above specified class change has occured

labled_array, number_of_objects = measurements.label(this_change)
name_point_list = np.unique(labled_array)
list_of_points=measurements.center_of_mass(this_change,labled_array,name_point_list)
output_list = [(b,a) for a,b in list_of_points]

/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/scipy/ndimage/measurements.py:1301: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]


In [10]:
output_list

[(nan, nan),
 (9041.0, 524.0),
 (9096.0, 958.0),
 (9013.0, 1015.0),
 (9607.0, 1186.0),
 (10169.0, 1586.0),
 (7214.0, 2015.0),
 (10670.0, 2038.0),
 (14960.0, 2074.0),
 (7211.0, 2133.0),
 (10829.0, 2143.0),
 (10829.0, 2146.0),
 (7748.0, 2234.0),
 (7759.0, 2250.0),
 (7737.0, 2253.0),
 (7738.0, 2254.0),
 (7739.0, 2255.0),
 (7737.0, 2257.0),
 (7740.0, 2257.0),
 (7750.0, 2260.0),
 (7751.0, 2262.0),
 (7739.0, 2264.0),
 (7745.0, 2266.0),
 (7750.0, 2267.0),
 (7754.0, 2273.0),
 (7759.0, 2278.0),
 (7755.0, 2287.0),
 (7733.0, 2288.0),
 (7780.0, 2288.0),
 (7754.0, 2289.0),
 (7754.0, 2292.0),
 (7732.0, 2300.0),
 (7730.0, 2306.0),
 (7740.0, 2308.0),
 (7742.0, 2311.0),
 (7773.0, 2313.0),
 (7773.0, 2320.0),
 (13488.0, 2460.0),
 (13487.0, 2461.0),
 (13867.0, 2502.0),
 (13830.0, 2543.0),
 (13839.0, 2547.0),
 (13891.0, 2563.0),
 (11503.0, 2583.0),
 (13921.0, 2585.0),
 (13939.0, 2600.0),
 (13743.5, 2618.0),
 (13352.0, 2695.0),
 (13392.0, 2801.0),
 (12176.0, 2903.0),
 (13208.0, 3177.0),
 (12387.0, 3184.0),


In [11]:
new_DLCD_stack = {}
band_list = [1, 2, 3,]# 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
for i in band_list:
    dataset = load_gfif_xr('new_DLCD/DLCDv2_2016_new.tif',i)
    dataset.coords['year'] = i 
    name = 'band '+str(i)
    #dataset = da.from_array(dataset, chunks=(100,100))
    new_DLCD_stack[name] = dataset
    print('done did ',i)
    
new_DLCD_ord = xr.concat(new_DLCD_stack.values(), 'year')
new_DLCD_ord = new_DLCD_ord.sortby('year')

done did  1
done did  2
done did  3


In [12]:
old_DLCD_stack = {}
band_list = [1, 2, 3,]# 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
for i in band_list:
    dataset = load_gfif_xr("old_DLCD/DLCDv2_Comp_old.tif",i)
    dataset.coords['year'] = i 
    name = 'band '+str(i)
    #dataset = da.from_array(dataset, chunks=(100,100))
    old_DLCD_stack[name] = dataset
    print('done did ',i)
    
old_DLCD_ord = xr.concat(old_DLCD_stack.values(), 'year')
old_DLCD_ord = old_DLCD_ord.sortby('year')

done did  1
done did  2
done did  3


In [13]:
old_DLCD_ord

<xarray.DataArray (year: 3, dim_0: 14902, dim_1: 19161)>
array([[[0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0],
        ..., 
        [0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0]],

       [[0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0],
        ..., 
        [0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0]],

       [[0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0],
        ..., 
        [0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0]]], dtype=uint8)
Coordinates:
  * year     (year) int64 1 2 3
Dimensions without coordinates: dim_0, dim_1

In [14]:
#final_table =dict()

New_DLCD_dit = dict()
Old_DLCD_dit = dict()

count = 0

for x,y in output_list:

    
    if x > 0:
        #set up lat-long for ndwi/redlight rill to cycle through pot/landslide events lits
        tup_x = int(x)
        tup_y = int(y)
        #set up stuff for file output nameing
        x_string = str(tup_x)
        y_string = str(tup_y)
        count = count+1
        #count_val = str(count)
        #name_val = 'point # '+count_val
        title_string = 'x= ' + x_string + ', y= '+ y_string

        #prepare ndwi7 vis light record for single point over epoc
        pd_new = new_version.isel(dim_0=tup_y,dim_1=tup_x)#(latitude=tup_y,longitude=tup_x)
        pd_old = old_version.isel(dim_0=tup_y,dim_1=tup_x)#(latitude=tup_y,longitude=tup_x)
        new_np = np.array(pd_new)
        old_np = np.array(pd_old)

        New_DLCD_dit[count] = new_np
        Old_DLCD_dit[count] = old_np
        
        #pd_new.plot.line('r-^')
        #pd_old.plot.line('b-^')
        #plt.show()
        #plt.close()
        
        

In [15]:
new_years=[2001, 2002 ,2003]#, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]
old_years=[2001, 2002 ,2003]#, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]

new_DLCD_panda = pd.DataFrame(New_DLCD_dit, index = new_years)
old_DLCD_panda = pd.DataFrame(Old_DLCD_dit, index = old_years)
new_DLCD_panda

,1,2,3,4,5,6,7,8,9,10,...,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050
2001,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2002,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2003,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [16]:
old_DLCD_panda

,1,2,3,4,5,6,7,8,9,10,...,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050
2001,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
2002,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
2003,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
